# Test 9 {#sec-test_9}
@fig-test_9, contains only current sources. The circuit is from @Hayt1978 (problem 20, Figure 2-39, page 61). Find the power absorbed by each source in the circuit. Added R1 to keep LTSpice happy.  

![test circuit](test_9.png){#fig-test_9}

The net list for the circuit was generated by LTSpice and show below:    
```
* test_9.asc
G1 0 1 1 0 0.6
I1 0 1 10
G2 1 0 1 0 0.8
R1 1 0 100Meg
.op
.backanno
.end
```

In [1]:
import os
from sympy import *
import numpy as np
from tabulate import tabulate
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
import SymMNA
from IPython.display import display, Markdown, Math, Latex
init_printing()

## Load the net list

In [2]:
net_list = '''
G1 0 1 1 0 0.6
I1 0 1 10
G2 1 0 1 0 0.8
*R1 1 0 100e6
'''

## Call the symbolic modified nodal analysis function

In [3]:
report, network_df, i_unk_df, A, X, Z = SymMNA.smna(net_list)

Display the equations

In [4]:
# reform X and Z into Matrix type for printing
Xp = Matrix(X)
Zp = Matrix(Z)
temp = ''
for i in range(len(X)):
    temp += '${:s}$<br>'.format(latex(Eq((A*Xp)[i:i+1][0],Zp[i])))

Markdown(temp)

$v_{1} \left(- g_{1} + g_{2}\right) = I_{1}$<br>

### Netlist statistics

In [5]:
print(report)

Net list report
number of lines in netlist: 3
number of branches: 3
number of nodes: 1
number of unknown currents: 0
number of RLC (passive components): 0
number of inductors: 0
number of independent voltage sources: 0
number of independent current sources: 1
number of op amps: 0
number of E - VCVS: 0
number of G - VCCS: 2
number of F - CCCS: 0
number of H - CCVS: 0
number of K - Coupled inductors: 0



### Connectivity Matrix

In [6]:
A

[-g₁ + g₂]

### Unknown voltages and currents

In [7]:
X

### Known voltages and currents

In [8]:
Z

### Network dataframe

In [9]:
network_df

,element,p node,n node,cp node,cn node,Vout,value,Vname,Lname1,Lname2
0,G1,0,1,1,0,NaN,0.6,NaN,NaN,NaN
1,I1,0,1,NaN,NaN,NaN,10.0,NaN,NaN,NaN
2,G2,1,0,1,0,NaN,0.8,NaN,NaN,NaN


### Unknown current dataframe

In [10]:
i_unk_df

,element,p node,n node


### Build the network equation matrix

In [11]:
# Put matrices into SymPy 
X = Matrix(X)
Z = Matrix(Z)

NE_sym = Eq(A*X,Z)
NE_sym

[v₁⋅(-g₁ + g₂)] = [I₁]

In [12]:
# turn the free symbols into SymPy variables
var(str(NE_sym.free_symbols).replace('{','').replace('}',''))

## Symbolic solution

In [13]:
U_sym = solve(NE_sym,X)

Display the symbolic solution

In [14]:
temp = ''
for i in U_sym.keys():
    temp += '${:s} = {:s}$<br>'.format(latex(i),latex(U_sym[i]))

Markdown(temp)

$v_{1} = - \frac{I_{1}}{g_{1} - g_{2}}$<br>

## Construct a dictionary of element values

In [15]:
element_value_keys = []
element_value_values = []

for i in range(len(network_df)):
    if network_df.iloc[i]['element'][0] == 'F' or network_df.iloc[i]['element'][0] == 'E' or network_df.iloc[i]['element'][0] == 'G' or network_df.iloc[i]['element'][0] == 'H':
        element_value_keys.append(var(network_df.iloc[i]['element'].lower()))
        element_value_values.append(network_df.iloc[i]['value'])
    else:
        element_value_keys.append(var(network_df.iloc[i]['element']))
        element_value_values.append(network_df.iloc[i]['value'])

element_values = dict(zip(element_value_keys, element_value_values))

element_values

## Numerical solution

In [16]:
NE = NE_sym.subs(element_values)
NE

[0.2⋅v₁] = [10.0]

In [17]:
U = solve(NE,X)

Display the numerical solution

Six significant digits are displayed so that results can be compared to LTSpice. 

In [18]:
table_header = ['unknown', 'mag']
table_row = []

for name, value in U.items():
    table_row.append([str(name),float(value)])

print(tabulate(table_row, headers=table_header,colalign = ('left','decimal'),tablefmt="simple",floatfmt=('5s','.6f')))

unknown          mag
---------  ---------
v1         50.000000


The node voltages and current through the sources are solved for. The Sympy generated solution matches the LTSpice results:
```
```
      --- Operating Point ---

V(1):	 50	 voltage
I(I1):	 10	 device_current
I(R1):	 5e-07	 device_current
I(G1):	 30	 device_current
I(G2):	 40	 device_current
```
The results from LTSpice agree with the SymPy results.

Find the power absorbed by each source in the circuit.  
```
The results from LTSpice agree with the SymPy results.

In [20]:
element_values[g1]*U[v1]**2 # power through G1

In [22]:
element_values[g2]*U[v1]**2 # power through G2

In [23]:
element_values[I1]*U[v1] # power through I1